In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
def remove_txt_headers(path):
    
    no_lines = ['Interval', 'ExcelDateTime', 'TimeFormat', 'DateFormat', 'ChannelTitle', 'Range', 'UnitName', 'TopValue', 
                'BottomValue']  # Lines to remove
    
    # Read raw lines
    with open(f'raw_labchart/{path}', 'r') as f:
        lines = f.readlines()
    
    # Remove lines
    lines_arr = np.array(lines)
    lines_arr = lines_arr[[not any(x in y for x in no_lines) for y in lines_arr]]

    # Write new file
    s = ''.join(lines_arr)
    with open('headers_removed/{0}_out.txt'.format(path.split('.')[0]), 'w') as f:
        f.write(s)

In [35]:
# Import trial data
dfs = [pd.read_excel('ap_data.xlsx', f'Sheet{x}').to_numpy() for x in range(1, 5)]

# Average trials
df = pd.DataFrame(np.mean(dfs, axis=0), columns = pd.read_excel('ap_data.xlsx').columns)

In [95]:
# Plot
fig = px.line(df, x='ml_amount', y='ap', color='voltage', width=1200, height=600)
fig.update_layout(
    title=dict(text='Action Potential in Response to Phenol Amount', x=0.5, font_size=40),
    xaxis=dict(title_text='Phenol Amount (mL)', title_font_size=30, tickfont_size=20),
    yaxis=dict(title_text='Average Response', title_font_size=30, tickfont_size=20),
    legend=dict(title_text='Stimulation Voltage (V)', font_size=20)
)
fig.update_traces(
    line=dict(width=7, dash='dash')
)

In [13]:
def plot_ap_sig(df, title):
    fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3])
    fig.add_trace(
        go.Scatter(x=df['time'], y=df['signal'], marker_opacity=0),
        row=1,
        col=1
    )

    fig.add_trace(
        go.Scatter(x=df['time'], y=df['stimulus'], marker_opacity=0),
        row=2,
        col=1
    )
    
    fig.update_yaxes(title_text='Voltage', row=1, col=1)

    fig.update_layout(
        width=800,
        height=600,
        title=dict(text=title, x=0.5, font_size=40),
        xaxis=dict(title_text='Time (s)', title_font_size=30, tickfont_size=20),
        yaxis=dict(title_text='Voltage (μV)', title_font_size=30, tickfont_size=20)
    )
    return fig

ap_dir = 'select_ap_signals'

no_ap = pd.read_excel(f'{ap_dir}/trial11.xlsx')
no_ap_fig = plot_ap_sig(no_ap, 'No Action Potential')
no_ap_fig.show()